Trying to make regressor with our dataset

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import PIL
import requests
import seaborn as sns
import pandas as pd
import scipy
import os
from glob import glob

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [4]:
os.getcwd()

'/Users/christineromano/DSI_Course/DSI_Capstone_Project'

In [5]:
cleaned_df = pd.read_excel('ttc-streetcar-delay-data-2024_cleaned.xlsx', index_col=None)
cleaned_df.head()

,Date,Line,Time,Day,Location,Incident,Min Delay,Min Gap,Bound,Vehicle,Month,Week,Day of Month,Hour of Day,Season
0,2024-04-28,301,03:54,Sunday,WOLSELEY LOOP,Cleaning - Unsanitary,30,60,E,8118,4,18,28,3,Spring
1,2024-09-14,301,02:23,Saturday,WOLSELEY LOOP,Utilized Off Route,30,60,W,8112,9,37,14,2,Summer
2,2024-01-28,301,02:11,Sunday,WARDEN AND COMSTOCK,Mechanical,10,20,W,8734,1,5,28,2,Winter
3,2024-11-15,301,02:35,Friday,THE QUEENSWAY AND WIND,Security,24,39,W,4588,11,46,15,2,Fall
4,2024-08-25,301,03:11,Sunday,THE QUEENSWAY AND GLEN,Mechanical,10,30,E,4569,8,35,25,3,Summer


In [6]:
# We have decided to drop some of these columns for analysis.  The "Date" column is subdivided into additional columns and can be removed, 
# the "Location" column will be removed due to lack of cleaned data, the "Time" column will be removed as it is substituted by "Hour of Day"

# Will also drop any blank values, as required for linear regression

cleaned_df_dropped = cleaned_df.drop(columns = ['Date', 'Time', 'Location'])
cleaned_df_dropped = cleaned_df_dropped.dropna()
cleaned_df_dropped.head()

,Line,Day,Incident,Min Delay,Min Gap,Bound,Vehicle,Month,Week,Day of Month,Hour of Day,Season
0,301,Sunday,Cleaning - Unsanitary,30,60,E,8118,4,18,28,3,Spring
1,301,Saturday,Utilized Off Route,30,60,W,8112,9,37,14,2,Summer
2,301,Sunday,Mechanical,10,20,W,8734,1,5,28,2,Winter
3,301,Friday,Security,24,39,W,4588,11,46,15,2,Fall
4,301,Sunday,Mechanical,10,30,E,4569,8,35,25,3,Summer


In [7]:
cleaned_df_dropped.dtypes

Line             int64
Day             object
Incident        object
Min Delay        int64
Min Gap          int64
Bound           object
Vehicle          int64
Month            int64
Week             int64
Day of Month     int64
Hour of Day      int64
Season          object
dtype: object

In [8]:
# Use one hot encoder to encode categorical variables

encoded_df = pd.get_dummies(cleaned_df_dropped, columns=['Line', 'Vehicle', 'Day', 'Incident', 'Bound', 'Season'], drop_first=True)
encoded_df

,Min Delay,Min Gap,Month,Week,Day of Month,Hour of Day,Line_303,Line_304,Line_305,Line_306,...,Incident_Overhead,Incident_Rail/Switches,Incident_Security,Incident_Utilized Off Route,Bound_N,Bound_S,Bound_W,Season_Spring,Season_Summer,Season_Winter
0,30,60,4,18,28,3,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
1,30,60,9,37,14,2,False,False,False,False,...,False,False,False,True,False,False,True,False,True,False
2,10,20,1,5,28,2,False,False,False,False,...,False,False,False,False,False,False,True,False,False,True
3,24,39,11,46,15,2,False,False,False,False,...,False,False,True,False,False,False,True,False,False,False
4,10,30,8,35,25,3,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13919,0,0,4,17,22,15,False,False,False,False,...,False,False,False,False,False,False,True,True,False,False
13920,8,15,2,8,18,19,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
13921,10,20,4,16,19,20,False,False,False,False,...,False,False,False,False,False,False,True,True,False,False
13922,4,8,2,8,20,16,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [9]:
X = encoded_df.drop(columns = 'Min Delay')
y = encoded_df['Min Delay']

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.75, random_state=42)

print(X_train.shape[0])
print(y_train.shape[0])

8964
8964


In [10]:
lm = LinearRegression()

In [11]:
lm.fit(X_train, y_train)

# Create a DataFrame containing the slope (coefficients) and intercept
coefficients_df = pd.DataFrame({
    "predictor": encoded_df.drop(columns = 'Min Delay').columns,
    "slope": lm.coef_,
    "intercept": [lm.intercept_] * len(lm.coef_)
})

# Display the coefficients DataFrame
print(coefficients_df)

r2 = lm.score(X_train, y_train)
print(r2)

# lm.coef_ gives the coefficients for each predictor (change in miles per gallon per unit change in each predictor variable)
# lm.intercept_ gives the intercept b_0 (the predicted miles per gallon when all predictors are set to 0)

          predictor     slope  intercept
0           Min Gap  0.921660 -16.337067
1             Month  4.972848 -16.337067
2              Week -1.134046 -16.337067
3      Day of Month  0.167866 -16.337067
4       Hour of Day -0.005645 -16.337067
...             ...       ...        ...
1161        Bound_S -0.118780 -16.337067
1162        Bound_W -0.248879 -16.337067
1163  Season_Spring  0.347555 -16.337067
1164  Season_Summer -0.300352 -16.337067
1165  Season_Winter  0.080325 -16.337067

[1166 rows x 3 columns]
0.946126648367495


In [12]:
y_pred = lm.predict(X_test)

In [13]:
rmpse = mean_squared_error(
    y_test, y_pred
)**0.5
rmpse

6.882642283796178

In [14]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
rmse

6.882642283796178